## <font style="font-family:roboto;color:#455e6c"> Introduction to atomistic simulations with pyiron </font>  

<div class="admonition note" name="html-admonition" style="background:#e3f2fd; padding: 10px">
<font style="font-family:roboto;color:#455e6c"> <b> DPG Tutorial: Creating and Running Automated Workflows for Material Science Simulations </b> </font> </br>
<font style="font-family:roboto;color:#455e6c"> 17 March 2024 </font>
</div>

Before the excercise, you should:

* Be familiar with python especially with numerical libraries like numpy and plotting tools like matplotlib
* Understand how Jupyter Notebooks work

The aim of this exercise is to make you familiar with:

* A general overview of what pyiron can do
* How to set up atomic structures and run atomistic simulation codes through pyiron

### <font style="font-family:roboto;color:#455e6c"> Importing necessary libraries </font>  
As a first step we import the libraries [numpy](http://www.numpy.org/) for data analysis and [matplotlib](https://matplotlib.org/) for visualization.

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt

Fundamentally, we only need to import one module from `pyiron`: the `Project` class

In [ ]:
from pyiron import Project

The Project object introduced below is central in pyiron. It allows to name the project as well as to derive all other objects such as structures, jobs etc. without having to import them. Thus, by code completion *Tab* the respective commands can be found easily.

We now create a pyiron Project named 'tutorial'.

### <font style="font-family:roboto;color:#455e6c"> Working with atomistic structures </font>  

#### <font style="font-family:roboto;color:#455e6c"> Creation of a project instance </font>  

In [ ]:
pr = Project("tutorial")

The project name also applies for the directory that is created for the project. All data generated by this `Project` object resides in this directory.

In [ ]:
pr.path

In [ ]:
pr

The `groups` and `nodes` will be populated later, as we add jobs and sub project to it.

#### <font style="font-family:roboto;color:#455e6c"> Creating atomic structures </font>  

Every atomistic simulation needs an atomic structure. For more details on generating and manipulating structures, please have a look at our [structures example](https://pyiron.readthedocs.io/en/latest/source/notebooks/structures.html). In this section however, we show how to generate and manipulate bulk crystals, surfaces, etc. pyiron's structure class is derived from the popular [`ase` package](https://wiki.fysik.dtu.dk/ase/ase/build/build.html) and any `ase` function to manipulate structures can also be applied here.

Creating a bulk fcc cubic unitcell

In [ ]:
Al_unitcell_cubic = pr.create.structure.bulk('Al', cubic=True, a=4.04)
Al_unitcell_cubic

Creating a super cell.

In [ ]:
Al_supercell_3_3_3 = Al_unitcell_cubic.repeat([3, 3, 3])
Al_supercell_3_3_3.plot3d(particle_size=2)

Creating a vacancy is easy, just delete an atom.

In [ ]:
Al_vacancy = Al_supercell_3_3_3.copy()
del Al_vacancy[0] # Deleting the first atom
print(Al_supercell_3_3_3.get_chemical_formula(), Al_vacancy.get_chemical_formula())
Al_vacancy.plot3d(particle_size=2)

#### <font style="font-family:roboto;color:#455e6c"> Creating random crystals </font>  

`pyxtal` is a [program](https://pyxtal.readthedocs.io/en/latest/) to generate random structures of a given space group and stoichiometry.
It is useful for crystal structure prediction and also for generating training structures.
We will use it later again, so let's briefly look at how it works.

In [ ]:
import structuretoolkit as stk
from pyiron_atomistics import ase_to_pyiron

In [ ]:
from pyiron_atomistics.atomistics.structure.structurestorage import StructureStorage

`structuretoolkit` is a library for structure manipulation and analysis also developed by the pyiron team.  For compatibility with a wider range of codes it operates purely on ASE `Atoms` objects, so we need to convert structures explicitely here.  In the next release of `pyiron_atomistics` you will be able to call `pr.create.structure.pyxtal` directly for a more convenient wrapper.

In [ ]:
fcc = ase_to_pyiron(stk.build.pyxtal(
    group=225,       # fcc
    species=['Al'],  # list of atoms we want to place
    num_ions=[4],    # how of each of them
))

fcc

In [ ]:
fcc.plot3d()

In [ ]:
groups = [1, 194, 225]

In [ ]:
store = StructureStorage()
for structure in stk.build.pyxtal(
    group=groups,
    species=['Al'],
    num_ions=[4]
):
    store.add_structure(structure['atoms'])

In [ ]:
structure = store.get_structure(1)
structure.plot3d()

### <font style="font-family:roboto;color:#455e6c"> Running an atomistic calculation using interatomic potentials (with LAMMPS) </font>  


Once we have an atomic structure, we can set up a simulation "job" of any atomistic simulation that is intergrated within pyiron. In this section, we are going to use the popular [LAMMPS code](https://lammps.sandia.gov/).

In [ ]:
# Create a job
job_lammps = pr.create.job.Lammps("lammps_job")

Every atomistic simulation code needs an input atomic structure. We use the Al supercell structure we created earlier

In [ ]:
# Assign an atomic structure to the job
job_lammps.structure = pr.create.structure.bulk('Al', cubic=True).repeat(3)

Once the structure is assigned, an appropriate potential should also be chosen. This list of available for the structure containing Al can be found below.  This list originates from the NIST Interatomic Potential Database.

In [ ]:
# See available potentials
job_lammps.list_potentials()[20:30]

In [ ]:
# Choose one of these potentials
job_lammps.potential = "2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1"

At this stage, the computational parameters for the simulation needs to be specified. pyiron parses generic computational parameters into code specific parameters allowing for an easy transition between simulation codes

In [ ]:
# specify calculation details: in this case: MD at 800 K in the NPT ensemble (pressure=0) for 10000 steps
job_lammps.calc_md(temperature=800, pressure=0, n_ionic_steps=10000)

We can now see how pyiron sets-up the corresponding LAMMPS input

In [ ]:
job_lammps.input.control

Once the `run()` commmand is called, pyiron creates necessary input files, calls the simulation code, and finally parses and stores the output.

In [ ]:
job_lammps.run()

When printing the project, the saved job will also appear under `nodes` now.

In [ ]:
pr

You can get a quick overview with the `job_table` method.

In [ ]:
pr.job_table()

Once it is finished we can access the parsed output.

In [ ]:
job_lammps['output']

In [ ]:
plt.plot(job_lammps['output/generic/energy_pot'])

### <font style="font-family:roboto;color:#455e6c"> Collecting structures and energies for training </font>  

To train a machine learning potential one usually starts with a set of structures, then run static DFT on each of them.  We will simulate this here, by running `lammps` with an existing potential to save some time.

Let's start by adding structures to our container.

In [ ]:
store = StructureStorage()

A usual approach is to start with some prototypical structures and strain and shake them.  We can use the fcc Al we created earlier.

In [ ]:
fcc

In [ ]:
strains = np.linspace(-0.05, 0.05, 5)
strains

In [ ]:
for eps in strains:
    store.add_structure(
        structure=fcc.apply_strain(eps, return_box=True),
        identifier=f'fcc_{eps}'
    )

In [ ]:
fcc.rattle?

In [ ]:
for eps in strains:
    structure = fcc.apply_strain(eps, return_box=True)
    structure.rattle(0.05)
    print('Strain', eps, 'Volume', structure.get_volume(per_atom=True))
    store.add_structure(
        structure=structure,
        identifier=f'fcc_{eps}_rattle'
    )

We usually also use random crystals generated by sampling random space groups.  One would use more spacegroups and differently sized unit cells, but we'll keep it simple here.  Details can be found in this paper.

https://journals.aps.org/prb/abstract/10.1103/PhysRevB.107.104103

In [ ]:
for i, structure in enumerate(stk.build.pyxtal(
    group=groups,
    species=['Al'],
    num_ions=[4],
    tm='metallic'
)):
    store.add_structure(
        structure=structure['atoms'],
        identifier=f'random_{i}'
    )

Now we can run our "expensive DFT".

In [ ]:
store.number_of_structures

In [ ]:
for i, structure in enumerate(store.iter_structures()):
    # pyiron has some opinions of what is a proper "job name" for technical reasons, but
    # you can always use this function to translate your favorite one into a "proper" one
    name = pr.create.job_name(store['identifier', i])
    job = pr.create.job.Lammps(name)
    job.potential = "2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1"
    job.structure = structure
    job.calc_static()
    job.run()

Once our reference calculations are finished, we can collect the results in a container for convenience.

In [ ]:
train = pr.create.job.TrainingContainer("Aluminium")

In [ ]:
train.include_structure?

In [ ]:
train.include_job?

In [ ]:
for job in pr.iter_jobs(hamilton='Lammps', status='finished'):
    for i in range(job.number_of_structures):
        train.include_job(job, iteration_step=i)

"Running" the container simply saves it to disk and to our database.  It can also precompute
nearest neighbor information.

In [ ]:
train.run()

Besides keeping everything in one place, the `TrainingContainer` also defines a number of plotting functions.

In [ ]:
df = train.plot.energy_volume()

In [ ]:
train.plot.energy_distance()
plt.xlabel(r'Minimum Nearest Neighbor Distance [$\mathrm{\AA}$]')
plt.ylabel('Energy [eV/atom]')

For fitting potentials within pyiron you can use the container as is, but if you have external tools, you can
export the data into a table.

In [ ]:
train.to_pandas()

### <font style="font-family:roboto;color:#455e6c"> Software used in this notebook </font>  

- [pyiron_atomistics](https://github.com/pyiron/pyiron_atomistics)
- [LAMMPS](https://www.lammps.org/)
- [pyXtal](https://pyxtal.readthedocs.io/en/latest/)